# Music Recommendation Project

Short description here... this will be Data Pre-processing: Import, exploration, cleaning, feature engineering, resulting in saving off feature data ... [data available on Kaggle](https://www.kaggle.com/c/kkbox-music-recommendation-challenge/data)

Author: Ben Walsh \
February 7, 2020

## Contents

1. [Data Import](#data-import)
2. Data Exploration
3. Data Cleaning
4. Feature Engineering
5. Saving Data

## <a class="anchor" id="data-import"></a>1. Data Import

### Import libraries

In [1]:
import pandas as pd
import os

First import all data: input traiing data, memeber data, and song data

In [18]:
member_datafile = './data-input/members.csv'
song_datafile = './data-input/songs.csv'

### Import member data

In [13]:
if os.path.exists(member_datafile):
    member_data = pd.read_csv(member_datafile)
else:
    print('Member data file {} not found!'.format(member_datafile))

### Import song data

In [19]:
if os.path.exists(song_datafile):
    song_data = pd.read_csv(song_datafile)
else:
    print('Song data file {} not found!'.format(song_datafile))

## 2. Data Exploration

### Member Data

In [20]:
member_data.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [21]:
member_data.describe()

,city,bd,registered_via,registration_init_time,expiration_date
count,34403.000000,34403.000000,34403.000000,3.440300e+04,3.440300e+04
mean,5.371276,12.280935,5.953376,2.013994e+07,2.016901e+07
std,6.243929,18.170251,2.287534,2.954015e+04,7.320925e+03
min,1.000000,-43.000000,3.000000,2.004033e+07,1.970010e+07
25%,1.000000,0.000000,4.000000,2.012103e+07,2.017020e+07
50%,1.000000,0.000000,7.000000,2.015090e+07,2.017091e+07
75%,10.000000,25.000000,9.000000,2.016110e+07,2.017093e+07
max,22.000000,1051.000000,16.000000,2.017023e+07,2.020102e+07


In [39]:
print('Member data # of data points: {}'.format(member_data.shape[0]))
print('Member data # of columns: {}'.format(member_data.shape[1]))
print('Number of unique cities that members come from: {}'.format(len(member_data['city'].unique())))
print('Percent of gender values that are empty: {:.2f}'.format(100*member_data['gender'].isna().sum()/len(member_data['gender'])))


Member data # of data points: 34403
Member data # of columns: 7
Number of unique cities that members come from: 21
Percent of gender values that are empty: 57.85


#### Observations

Gender values are mostly empty. With such a large proportion missing, removing the corresponding NaN entries would remove a large portion of the data. Recommend removing gender as a feature.

### Song Data

In [40]:
song_data.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [41]:
song_data.describe()

,song_length,language
count,2.296320e+06,2.296319e+06
mean,2.469935e+05,3.237800e+01
std,1.609200e+05,2.433241e+01
min,1.850000e+02,-1.000000e+00
25%,1.836000e+05,-1.000000e+00
50%,2.266270e+05,5.200000e+01
75%,2.772690e+05,5.200000e+01
max,1.217385e+07,5.900000e+01


In [42]:
print('Song data # of data points: {}'.format(song_data.shape[0]))
print('Song data # of columns: {}'.format(song_data.shape[1]))
print('Number of unique languages that songs come from: {}'.format(len(song_data['language'].unique())))

Song data # of data points: 2296320
Song data # of columns: 7
Number of unique languages that songs come from: 11


In [83]:
[ genre_id.find('|') for genre_id in song_data['genre_ids'].iloc[:10].values[:10]]

[-1, -1, -1, -1, -1, 3, -1, -1, -1, 3]